In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[*]") \
        .appName('Yellow2024') \
        .getOrCreate()

25/03/06 21:22:08 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/06 21:22:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/06 21:22:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/06 21:22:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/06 21:22:11 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
df = spark.read.parquet("yellow_tripdata_2024-10.parquet")

In [3]:
df_repartitioned = df.repartition(4)

In [5]:
df_repartitioned.write.parquet('data/pq1', mode='overwrite')

In [7]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [11]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [8]:
from pyspark.sql import functions as F

In [27]:
# result = spark.sql(
#     """
#     SELECT Count(*)
#     FROM yellow_2024
#     WHERE DATE(tpep_pickup_datetime) = '2024-10-15' 
#     """
# )
# result.show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [28]:
from pyspark.sql.functions import *
df_filter = df.filter(to_date(col("tpep_pickup_datetime")) == '2024-10-15')
trip_count = df_filter.count()
trip_count

128893

In [45]:
df_duration = df.withColumn("duration", 
                            (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600
                            )

In [46]:
max_duration = df_duration.agg({"duration": "max"}).show()

+------------------+
|     max(duration)|
+------------------+
|162.61777777777777|
+------------------+



In [50]:
zones_df = spark.read.csv("taxi_zone_lookup.csv", header=True)
zones_df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [68]:
pickup_counts = df.groupBy("PULocationID").count()

In [69]:
joined_df = pickup_counts.join(zones_df, zones_df.LocationID == df.PULocationID)

In [55]:
# joined_df.createOrReplaceTempView('yellow_data')

In [77]:
joined_df.select("zone", "count").orderBy(col("count").asc()).show()

+--------------------+-----+
|                zone|count|
+--------------------+-----+
|Governor's Island...|    1|
|       Arden Heights|    2|
|       Rikers Island|    2|
|         Jamaica Bay|    3|
| Green-Wood Cemetery|    3|
|Charleston/Totten...|    4|
|Eltingville/Annad...|    4|
|       West Brighton|    4|
|   Rossville/Woodrow|    4|
|       Port Richmond|    4|
|         Great Kills|    6|
|        Crotona Park|    6|
|     Mariners Harbor|    7|
|Heartland Village...|    7|
|Saint George/New ...|    9|
|             Oakwood|    9|
|New Dorp/Midland ...|   10|
|       Broad Channel|   10|
|         Westerleigh|   12|
|     Pelham Bay Park|   12|
+--------------------+-----+
only showing top 20 rows

